In [ ]:
!pip install h5py
!pip install pandas

In [13]:
#reading h5
import h5py

# Define the file path to the .h5 file
file_path = '/content/GSM7021712_C_1m_raw_feature_bc_matrix.h5'

# Open the .h5 file
with h5py.File(file_path, 'r') as f:
    # Access the 'matrix' group
    matrix_group = f['matrix']

    # List all datasets within 'matrix'
    print("Keys in 'matrix':", list(matrix_group.keys()))

    # Access and print information about each dataset
    for key in matrix_group:
        item = matrix_group[key]
        if isinstance(item, h5py.Dataset):
            print(f"Dataset '{key}' shape: {item.shape}")
            print(f"Dataset '{key}' dtype: {item.dtype}")
            # Print the first few elements of the dataset if it's not too large
            try:
                print(f"First few elements of '{key}':", item[:10])  # Adjust slicing as needed
            except Exception as e:
                print(f"Could not read data from '{key}': {e}")
        elif isinstance(item, h5py.Group):
            print(f"Group '{key}'")
        else:
            print(f"Unknown item type for '{key}'")


Keys in 'matrix': ['barcodes', 'data', 'features', 'indices', 'indptr', 'shape']
Dataset 'barcodes' shape: (6794880,)
Dataset 'barcodes' dtype: |S24
First few elements of 'barcodes': [b'AAACCCAAGAAACACT-1' b'AAACCCAAGAAACCAT-1' b'AAACCCAAGAAACCCA-1'
 b'AAACCCAAGAAACCCG-1' b'AAACCCAAGAAACCTG-1' b'AAACCCAAGAAACGAA-1'
 b'AAACCCAAGAAACGTC-1' b'AAACCCAAGAAACTAC-1' b'AAACCCAAGAAACTCA-1'
 b'AAACCCAAGAAACTGC-1']
Dataset 'data' shape: (25118049,)
Dataset 'data' dtype: int32
First few elements of 'data': [1 1 1 1 1 1 1 1 1 1]
Group 'features'
Dataset 'indices' shape: (25118049,)
Dataset 'indices' dtype: int64
First few elements of 'indices': [ 3141  3919 12168 20212 24842 30005 32201 12185 29544 32201]
Dataset 'indptr' shape: (6794881,)
Dataset 'indptr' dtype: int64
First few elements of 'indptr': [0 0 7 8 8 8 8 8 8 9]
Dataset 'shape' shape: (2,)
Dataset 'shape' dtype: int32
First few elements of 'shape': [  32285 6794880]


In [14]:
#download feature(species, geneID and name)
import h5py
import pandas as pd
import numpy as np

# Define the file path to the .h5 file
file_path = '/content/GSM7021712_C_1m_raw_feature_bc_matrix'

# Define the path for the combined CSV output
output_csv_path = '/content/features_combined.csv'

# Open the .h5 file
with h5py.File(file_path, 'r') as f:
    # Access the 'matrix' group
    matrix_group = f['matrix']

    # Check if 'features' group exists
    if 'features' in matrix_group:
        features_group = matrix_group['features']

        # List of datasets to combine
        dataset_keys = ['_all_tag_keys', 'feature_type', 'genome', 'id', 'name']

        # Initialize a dictionary to hold DataFrames
        data_dict = {}

        for key in dataset_keys:
            if key in features_group:
                dataset = features_group[key]
                data = dataset[:]

                # Convert data to a DataFrame
                if dataset.dtype.char == 'S':  # If the data type is a string
                    data = [item.decode('utf-8') for item in data]
                df = pd.DataFrame(data, columns=[key])

                # Add DataFrame to dictionary
                data_dict[key] = df
            else:
                print(f"Dataset '{key}' not found in 'features'.")

        # Concatenate all DataFrames horizontally
        if data_dict:
            combined_df = pd.concat(data_dict.values(), axis=1)
            combined_df.to_csv(output_csv_path, index=False)
            print(f"Combined feature data saved to CSV as '{output_csv_path}'")
        else:
            print("No datasets found to combine.")
    else:
        print("'features' group not found in 'matrix'.")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/GSM7021712_C_1m_raw_feature_bc_matrix', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [1]:
# based on result above add raw count and CPM
!pip install scipy

In [9]:
#debug
import h5py

# Define the file path to the .h5 file
file_path = '/content/GSM7021712_C_1m_raw_feature_bc_matrix.h5'

# Open the .h5 file
with h5py.File(file_path, 'r') as f:
    # Access the 'matrix' group
    matrix_group = f['matrix']

    # Print all datasets in the matrix group
    print("Datasets in the 'matrix' group:")
    for name in matrix_group:
        print(name)

    # Access data arrays for raw counts
    data_matrix = matrix_group['data'][:]  # Non-zero values
    indices = matrix_group['indices'][:]    # Row indices
    indptr = matrix_group['indptr'][:]      # Index pointer

    # Print the lengths of the arrays
    print(f"Data length: {len(data_matrix)}")
    print(f"Indices length: {len(indices)}")
    print(f"Indptr length: {len(indptr)}")

    # Print the first few entries of indptr to see its structure
    print("First few entries of indptr:", indptr[:10])


Datasets in the 'matrix' group:
barcodes
data
features
indices
indptr
shape
Data length: 25118049
Indices length: 25118049
Indptr length: 6794881
First few entries of indptr: [0 0 7 8 8 8 8 8 8 9]


In [16]:
#feature(gene name id species)

import h5py
import pandas as pd

# Define the file path to the .h5 file
file_path = '/content/GSM7021712_C_1m_raw_feature_bc_matrix.h5'

# Open the .h5 file
with h5py.File(file_path, 'r') as f:
    # Access the 'matrix' group
    matrix_group = f['matrix']

    # Access the features group
    features_group = matrix_group['features']

    # Access individual datasets within the features group
    feature_type = features_group['feature_type'][:]  # Feature types
    genome = features_group['genome'][:]              # Genome information
    ids = features_group['id'][:]                      # IDs
    names = features_group['name'][:]                  # Names

    # Convert data to lists and decode
    feature_type = [item.decode('utf-8') for item in feature_type]
    genome = [item.decode('utf-8') for item in genome]
    ids = [item.decode('utf-8') for item in ids]
    names = [item.decode('utf-8') for item in names]

    # Create a DataFrame for features without '_all_tag_keys'
    features_df = pd.DataFrame({
        'feature_type': feature_type,
        'genome': genome,
        'id': ids,
        'name': names
    })

    # Print the DataFrame to verify
    print(features_df.head())


      feature_type genome                  id     name
0  Gene Expression   mm10  ENSMUSG00000051951     Xkr4
1  Gene Expression   mm10  ENSMUSG00000089699   Gm1992
2  Gene Expression   mm10  ENSMUSG00000102331  Gm19938
3  Gene Expression   mm10  ENSMUSG00000102343  Gm37381
4  Gene Expression   mm10  ENSMUSG00000025900      Rp1


In [24]:
#debug
import h5py
import pandas as pd
from scipy.sparse import csr_matrix

# Define the file path to the .h5 file
file_path = '/content/GSM7021712_C_1m_raw_feature_bc_matrix.h5'

# Open the .h5 file
with h5py.File(file_path, 'r') as f:
    # Access the 'matrix' group
    matrix_group = f['matrix']

    # Access the features group
    features_group = matrix_group['features']

    # Access individual datasets within the features group
    feature_type = features_group['feature_type'][:]  # Feature types
    genome = features_group['genome'][:]              # Genome information
    ids = features_group['id'][:]                      # IDs
    names = features_group['name'][:]                  # Names

    # Convert data to lists and decode
    feature_type = [item.decode('utf-8') for item in feature_type]
    genome = [item.decode('utf-8') for item in genome]
    ids = [item.decode('utf-8') for item in ids]
    names = [item.decode('utf-8') for item in names]

    # Create a DataFrame for features
    features_df = pd.DataFrame({
        'feature_type': feature_type,
        'genome': genome,
        'id': ids,
        'name': names
    })

    # Access raw counts data
    data = matrix_group['data'][:]        # Non-zero values
    indices = matrix_group['indices'][:]  # Row indices
    indptr = matrix_group['indptr'][:]    # Index pointer
    shape = matrix_group['shape'][:]       # Shape of the matrix

    # Print shapes for debugging
    print(f"Data shape: {data.shape}")
    print(f"Indices shape: {indices.shape}")
    print(f"Indptr shape: {indptr.shape}")
    print(f"Shape of the matrix: {shape}")

    # Print some values for debugging
    print("\nIndptr values (first 10):", indptr[:10])
    print("Data values (first 10):", data[:10])
    print("Indices values (first 10):", indices[:10])

    # Create a sparse matrix for raw counts
    try:
        raw_counts = csr_matrix((data, indices, indptr), shape)
    except ValueError as e:
        print("Error creating sparse matrix:", e)
        print("Length of indptr:", len(indptr))
        print("Expected number of rows:", shape[0] + 1)

    # If sparse matrix creation is successful, proceed with conversion to dense
    else:
        # Convert raw counts to a dense format (optional)
        raw_counts_dense = raw_counts.toarray()

        # Calculate CPM
        total_counts = raw_counts.sum(axis=1).A1  # Sum across columns for each row (cell)
        cpm = (raw_counts.multiply(1e6 / total_counts[:, None])).toarray()  # CPM calculation

        # Create a DataFrame for raw counts
        raw_counts_df = pd.DataFrame(raw_counts_dense, columns=features_df['name'])

        # Create a DataFrame for CPM
        cpm_df = pd.DataFrame(cpm, columns=features_df['name'])

        # Print the DataFrames to verify
        print("Features DataFrame:")
        print(features_df.head())

        print("\nRaw Counts DataFrame:")
        print(raw_counts_df.head())

        print("\nCPM DataFrame:")
        print(cpm_df.head())


#The output confirms that the indptr values do not match the expected size for creating the sparse matrix,
#which is leading to the error. Let's break down the issues observed from the output:

#Key Observations
#Shape of the Matrix: The shape is given as [32285, 6794880], indicating that you expect 32285 rows and 6794880 columns
#in the matrix.

#Indptr Length: The indptr length is 6794881, which should actually be 32286
#(one more than the number of rows) to define the matrix correctly.

#Indptr Values: The first few values of indptr show that many rows have no entries.
#Specifically, the values are [0, 0, 7, 8, 8, 8, 8, 8, 8, 9], suggesting that:

#The first two rows have no non-zero entries.
#The third row has 7 non-zero entries.
#The subsequent rows have very few entries, with most of them clustered around very low indices.
#Potential Issues and Solutions
#Empty Rows: The indptr values starting with 0, 0 suggest that the first row may be entirely empty,
#and this might have implications on how the data was structured or generated.

#Data Generation: The mismatch in the number of rows and the entries in the indptr could indicate an issue in how the data,
#indices, or indptr arrays were constructed.
#There might be a possibility that not all features (genes) have corresponding counts,
\#leading to a misalignment between the indices and the intended structure.





Data shape: (25118049,)
Indices shape: (25118049,)
Indptr shape: (6794881,)
Shape of the matrix: [  32285 6794880]

Indptr values (first 10): [0 0 7 8 8 8 8 8 8 9]
Data values (first 10): [1 1 1 1 1 1 1 1 1 1]
Indices values (first 10): [ 3141  3919 12168 20212 24842 30005 32201 12185 29544 32201]
Error creating sparse matrix: index pointer size (6794881) should be (32286)
Length of indptr: 6794881
Expected number of rows: 32286


In [17]:
# Assuming you have already opened the .h5 file as before
with h5py.File(file_path, 'r') as f:
    # Access raw counts and CPM (assuming they are stored in 'data' and 'CPM' datasets)
    raw_counts = matrix_group['data'][:]
    cpm = matrix_group['cpm'][:]  # If CPM is available, replace this line as needed

    # Create a DataFrame for raw counts
    raw_counts_df = pd.DataFrame(raw_counts, columns=features_df['id'])

    # Create a DataFrame for CPM
    cpm_df = pd.DataFrame(cpm, columns=features_df['id'])

    # Concatenate the DataFrames to include raw counts and CPM
    final_df = pd.concat([features_df, raw_counts_df, cpm_df], axis=1)

    # Save to CSV
    final_df.to_csv('gene_expression_data.csv', index=False)

# Print the final DataFrame to verify
print(final_df.head())


KeyError: 'Unable to synchronously open object (invalid identifier type to function)'

In [ ]:
import h5py
import numpy as np

# Define the file path to the .h5 file
file_path = '/content/GSM7021712_C_1m_raw_feature_bc_matrix.h5'

# Define terms to search for
search_terms = ['WT', 'wildtype']
search_terms = ['WT', 'wildtypeAPP']

def search_terms_in_dataset(dataset, terms):
    """Search for terms in a dataset."""
    if dataset.dtype.char == 'S':  # If the data type is a string
        data = [item.decode('utf-8') for item in dataset[:]]
        found_terms = [term for term in terms if any(term in item for item in data)]
        return found_terms
    return []

# Open the .h5 file
with h5py.File(file_path, 'r') as f:
    # Check the file-level attributes
    print("Checking file-level attributes...")
    for attr in f.attrs:
        if any(term in str(f.attrs[attr]) for term in search_terms):
            print(f"Term found in file-level attribute '{attr}': {f.attrs[attr]}")

    # Check groups and datasets
    def search_in_group(group, group_path=""):
        print(f"\nSearching in group '{group_path}':")
        for name, item in group.items():
            if isinstance(item, h5py.Group):
                search_in_group(item, group_path + "/" + name)
            elif isinstance(item, h5py.Dataset):
                found_terms = search_terms_in_dataset(item, search_terms)
                if found_terms:
                    print(f"Term(s) {found_terms} found in dataset '{group_path}/{name}'")

    # Start searching from the root
    search_in_group(f)


Checking file-level attributes...

Searching in group '':

Searching in group '/matrix':

Searching in group '/matrix/features':


In [ ]:
import h5py

# Define the file path to the .h5 file
file_path = '/content/GSM7021712_C_1m_raw_feature_bc_matrix.h5'

# Open the .h5 file
with h5py.File(file_path, 'r') as f:
    # Recursive function to list all groups and datasets
    def list_all_groups_and_datasets(group, group_path=""):
        print(f"Group '{group_path}':")
        for name, item in group.items():
            if isinstance(item, h5py.Group):
                print(f"  Group '{group_path}/{name}'")
                list_all_groups_and_datasets(item, group_path + "/" + name)
            elif isinstance(item, h5py.Dataset):
                print(f"  Dataset '{group_path}/{name}'")

    # Start listing from the root
    list_all_groups_and_datasets(f)


Group '':
  Group '/matrix'
Group '/matrix':
  Dataset '/matrix/barcodes'
  Dataset '/matrix/data'
  Group '/matrix/features'
Group '/matrix/features':
  Dataset '/matrix/features/_all_tag_keys'
  Dataset '/matrix/features/feature_type'
  Dataset '/matrix/features/genome'
  Dataset '/matrix/features/id'
  Dataset '/matrix/features/name'
  Dataset '/matrix/indices'
  Dataset '/matrix/indptr'
  Dataset '/matrix/shape'
